Speech Emotion Identification

Done by:

K Sai Sreelekha

19251A0590

GNITS

**Import required libraries**

In [1]:
import librosa
import soundfile
import pandas as pd
import os, glob, pickle
import numpy as np
from tqdm import tqdm
from scipy.io import wavfile
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

**Mount the dataset file for importing the dataset**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Loading the Dataset (Ravdess)**

In [4]:
os.listdir(path='drive/MyDrive/dataset')
def getListOfFiles(dirName):
    listOfFile=os.listdir(dirName)
    allFiles=list()
    for entry in listOfFile:
        fullPath=os.path.join(dirName, entry)
        if os.path.isdir(fullPath):
            allFiles=allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
    return allFiles

dirName = 'drive/MyDrive/dataset'
listOfFiles = getListOfFiles(dirName)
len(listOfFiles)

1569

**Cleaning the dataset**

DOWN SAMPLING OF AUDIO FILES IS DONE AND PUT MASK OVER IT AND DIRECT INTO CLEAN FOLDER
MASK IS TO REMOVE UNNECESSARY EMPTY VOIDES AROUND THE MAIN AUDIO VOICE

In [5]:
def envelope(y , rate, threshold):
    mask=[]
    y=pd.Series(y).apply(np.abs)
    y_mean = y.rolling(window=int(rate/10) ,  min_periods=1 , center = True).mean()
    for mean in y_mean:
        if mean>threshold:
            mask.append(True)
        else:
            mask.append(False)
    return mask

**Extract features (mfcc, chroma, mel) from a sound file**

In [6]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
        result=np.hstack((result, mel))
    return result

Emotions in the dataset

In [7]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
#These are the emotions User wants to observe more :
observed_emotions=['calm', 'happy', 'angry', 'disgust']

 **Loading the clean data and extracting the data from it**

In [8]:
from glob import glob
import os
import glob
def load_data(test_size=0.33):
    x,y=[],[]
    answer = 0
    for file in glob.glob('drive/MyDrive/dataset//Actor_*//*.wav'):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            answer += 1
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append([emotion,file_name])
    return train_test_split(np.array(x), y, test_size=test_size, random_state=0)

**Splitting the dataset into test and trial data**

In [9]:
x_train,x_test,y_train,y_test=load_data(test_size=0.25)
print(np.shape(x_train),np.shape(x_test), np.shape(y_train),np.shape(y_test))
y_test_map = np.array(y_test).T
y_test = y_test_map[0]
test_filename = y_test_map[1]
y_train_map = np.array(y_train).T
y_train = y_train_map[0]
train_filename = y_train_map[1]
print(np.shape(y_train),np.shape(y_test))
print(*test_filename,sep="\n")

(576, 180) (192, 180) (576, 2) (192, 2)
(576,) (192,)
03-01-03-01-01-01-01.wav
03-01-07-01-02-01-22.wav
03-01-02-01-02-02-22.wav
03-01-05-01-02-02-19.wav
03-01-07-01-02-01-05.wav
03-01-05-01-02-02-22.wav
03-01-03-01-02-01-14.wav
03-01-07-02-02-02-06.wav
03-01-03-02-01-01-11.wav
03-01-07-01-02-01-21.wav
03-01-07-02-01-02-19.wav
03-01-03-01-01-01-12.wav
03-01-07-01-01-01-23.wav
03-01-05-01-01-02-06.wav
03-01-02-02-01-02-03.wav
03-01-07-02-01-02-21.wav
03-01-03-02-01-02-17.wav
03-01-05-02-02-01-18.wav
03-01-05-01-01-02-07.wav
03-01-07-01-01-02-13.wav
03-01-02-02-01-02-08.wav
03-01-02-01-02-01-06.wav
03-01-02-02-01-02-17.wav
03-01-07-02-02-01-13.wav
03-01-07-01-02-01-19.wav
03-01-02-02-02-02-06.wav
03-01-02-02-01-01-15.wav
03-01-02-01-02-02-03.wav
03-01-02-01-02-02-24.wav
03-01-03-01-01-02-10.wav
03-01-07-02-02-01-23.wav
03-01-02-02-02-01-08.wav
03-01-05-01-02-02-01.wav
03-01-07-02-01-01-13.wav
03-01-05-01-01-01-11.wav
03-01-02-01-02-02-11.wav
03-01-02-01-01-01-01.wav
03-01-07-01-02-02-10.

**Initialize the Multi Layer Perceptron Classifier**

In [10]:
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

**Train the model**

In [11]:
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

**Predicting the y_test values using the loaded Model**

In [12]:
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy of the Recognizer is: {:.1f}%".format(accuracy*100))


Accuracy of the Recognizer is: 64.6%


**Saving the predictions in a .csv file**

In [13]:
import pandas as pd
y_pred1 = pd.DataFrame(y_pred , columns=['Predictions'])
y_pred1['Name of Files'] = test_filename
print(y_pred1)
y_pred1.to_csv('FinalPredictions.csv')

    Predictions             Name of Files
0         happy  03-01-03-01-01-01-01.wav
1         happy  03-01-07-01-02-01-22.wav
2          calm  03-01-02-01-02-02-22.wav
3         happy  03-01-05-01-02-02-19.wav
4          calm  03-01-07-01-02-01-05.wav
..          ...                       ...
187       happy  03-01-02-02-02-02-13.wav
188       angry  03-01-05-01-01-01-16.wav
189       happy  03-01-05-01-01-02-13.wav
190       happy  03-01-03-02-01-02-20.wav
191       happy  03-01-03-01-01-02-07.wav

[192 rows x 2 columns]


In [16]:
y_pred1[:10]

,Predictions,Name of Files
0,happy,03-01-03-01-01-01-01.wav
1,happy,03-01-07-01-02-01-22.wav
2,calm,03-01-02-01-02-02-22.wav
3,happy,03-01-05-01-02-02-19.wav
4,calm,03-01-07-01-02-01-05.wav
5,happy,03-01-05-01-02-02-22.wav
6,happy,03-01-03-01-02-01-14.wav
7,disgust,03-01-07-02-02-02-06.wav
8,happy,03-01-03-02-01-01-11.wav
9,disgust,03-01-07-01-02-01-21.wav


In [15]:
from sklearn.metrics import f1_score

In [17]:
f1_score(y_test, y_pred,average=None)

array([0.69767442, 0.74074074, 0.52307692, 0.592     ])

In [18]:
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,happy,happy
1,disgust,happy
2,calm,calm
3,angry,happy
4,disgust,calm
5,angry,happy
6,happy,happy
7,disgust,disgust
8,happy,happy
9,disgust,disgust


In [19]:
import pickle
# Writing different model files to file
with open( 'modelForPrediction1.sav', 'wb') as f:
    pickle.dump(model,f)

**Test the model with any other data**

In [20]:
filename = 'modelForPrediction1.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature("/content/drive/MyDrive/archive/audio_speech_actors_01-24/Actor_01/03-01-01-01-01-01-01.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

array(['calm'], dtype='<U7')

In [ ]:
feature

In [25]:
filename = 'modelForPrediction1.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature("/content/drive/MyDrive/archive/audio_speech_actors_01-24/Actor_10/03-01-02-01-01-01-10.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

array(['happy'], dtype='<U7')

In [ ]:
feature